# Stripe Connect: On-Demand Delivery Platform

**A DoorDash-style marketplace demonstrating Stripe Connect fundamentals**

---

### Quick Navigation
| Section | Topic |
|---------|-------|
| [**1. Onboarding**](#section-1-onboarding-connected-accounts) | Create restaurant & courier accounts |
| [**2. Payment**](#section-2-collecting-customer-payment) | Customer pays $35 for an order |
| [**3. Transfers**](#section-3-routing-funds-with-transfers) | Split payment to multiple parties |
| [**4. Edge Cases**](#section-4-edge-cases--real-world-scenarios) | Refunds, tips, instant payouts |

---

### The Players

| Party | Role | Stripe Representation |
|-------|------|----------------------|
| **Platform** | Delivery service (us) | Main Stripe account |
| **Restaurant** | Provides food | Custom Connected Account |
| **Courier** | Delivers food | Custom Connected Account |
| **Customer** | Pays for order | Customer object |

### Money Flow ($35 order)

```
Customer pays $35.00
    │
    ▼
Platform Stripe Balance ($35.00)
    │
    ├── Transfer $25.00 ──► Restaurant (food cost)
    ├── Transfer $7.00  ──► Courier (delivery fee)
    └── Keep $3.00      ──► Platform (service fee)
```

---
# Setup

In [ ]:
import stripe
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../.env')

# Initialize Stripe with secret key
stripe.api_key = os.getenv('STRIPE_SECRET_KEY')

# Verify connection
account = stripe.Account.retrieve()
print(f"Connected to Stripe account: {account.id}")
print(f"Business name: {account.settings.dashboard.display_name or 'Not set'}")

---
# Section 1: Onboarding Connected Accounts

> **Goal:** Create Custom connected accounts for a restaurant and courier

### Why Custom Accounts?

For a delivery platform, we use **Custom** connected accounts because:
- **Full control** over the user experience (white-label onboarding)
- **Platform manages** communication with connected accounts
- **Flexible payout** timing and control
- Best for platforms where users don't need direct Stripe Dashboard access

### Onboarding Flow

```
Create Account → Collect KYC Info → Accept ToS → Verification → Enable Payouts
```

### 1.1 Create Restaurant Account

In [ ]:
# Create a Custom connected account for a restaurant
restaurant_account = stripe.Account.create(
    type="custom",
    country="US",
    email="restaurant@example.com",
    capabilities={
        "card_payments": {"requested": True},
        "transfers": {"requested": True},
    },
    business_type="company",
    business_profile={
        "name": "Mario's Pizza",
        "mcc": "5812",  # Restaurants
        "url": "https://mariospizza.example.com",
    },
    metadata={
        "platform_role": "restaurant",
        "internal_id": "rest_001"
    }
)

print(f"Restaurant Account ID: {restaurant_account.id}")
print(f"Payouts enabled: {restaurant_account.payouts_enabled}")
print(f"Charges enabled: {restaurant_account.charges_enabled}")

### 1.2 Create Onboarding Link

For Custom accounts, we use **Account Links** to redirect users to Stripe-hosted onboarding forms (recommended for compliance):

In [ ]:
# Create an Account Link for hosted onboarding
account_link = stripe.AccountLink.create(
    account=restaurant_account.id,
    refresh_url="https://yourplatform.com/onboarding/refresh",
    return_url="https://yourplatform.com/onboarding/complete",
    type="account_onboarding",
)

print(f"Onboarding URL: {account_link.url}")
print(f"\nThis link expires at: {account_link.expires_at}")
print("\nIn production, redirect the restaurant owner to this URL to complete onboarding.")

### 1.3 Create Courier Account

In [ ]:
# Create a Custom connected account for a courier (individual)
courier_account = stripe.Account.create(
    type="custom",
    country="US",
    email="courier@example.com",
    capabilities={
        "card_payments": {"requested": True},
        "transfers": {"requested": True},
    },
    business_type="individual",  # Couriers are typically individuals
    business_profile={
        "mcc": "4215",  # Courier services
        "url": "https://yourplatform.com/couriers/c001",
    },
    metadata={
        "platform_role": "courier",
        "internal_id": "cour_001"
    }
)

print(f"Courier Account ID: {courier_account.id}")
print(f"Payouts enabled: {courier_account.payouts_enabled}")
print(f"Charges enabled: {courier_account.charges_enabled}")

### 1.4 Check Account Requirements

Before an account can receive transfers, Stripe requires certain information for compliance:

In [ ]:
# Check what's needed for the restaurant account
account = stripe.Account.retrieve(restaurant_account.id)

print("=== Restaurant Account Requirements ===")
print(f"\nCurrently due: {account.requirements.currently_due}")
print(f"\nEventually due: {account.requirements.eventually_due}")
print(f"\nPending verification: {account.requirements.pending_verification}")

---
# Section 2: Collecting Customer Payment

> **Goal:** Create a $35 order and collect payment from the customer

### Payment Flow

```
Create Customer → Create PaymentIntent ($35) → Confirm with Payment Method
```

We're using **Separate Charges and Transfers**:
- Payment goes to platform's Stripe balance first
- Platform then transfers funds to connected accounts
- Gives platform full control over fund routing and timing

### 2.1 Create Customer

In [ ]:
# Create a customer for the order
customer = stripe.Customer.create(
    email="hungry.customer@example.com",
    name="Jane Smith",
    metadata={
        "platform_user_id": "user_12345"
    }
)

print(f"Customer ID: {customer.id}")
print(f"Customer email: {customer.email}")

### 2.2 Create PaymentIntent

**Order breakdown:**
| Item | Amount |
|------|--------|
| Food (subtotal) | $25.00 |
| Delivery fee | $7.00 |
| Platform fee | $3.00 |
| **Total** | **$35.00** |

In [ ]:
# Order amounts (in cents)
FOOD_COST = 2500      # $25.00 - goes to restaurant
DELIVERY_FEE = 700    # $7.00 - goes to courier
PLATFORM_FEE = 300    # $3.00 - platform keeps
TOTAL_AMOUNT = FOOD_COST + DELIVERY_FEE + PLATFORM_FEE  # $35.00

# Create PaymentIntent
payment_intent = stripe.PaymentIntent.create(
    amount=TOTAL_AMOUNT,
    currency="usd",
    customer=customer.id,
    metadata={
        "order_id": "order_98765",
        "restaurant_id": restaurant_account.id,
        "courier_id": courier_account.id,
        "food_cost": FOOD_COST,
        "delivery_fee": DELIVERY_FEE,
        "platform_fee": PLATFORM_FEE,
    },
    description="Order #98765 - Mario's Pizza",
)

print(f"PaymentIntent ID: {payment_intent.id}")
print(f"Amount: ${payment_intent.amount / 100:.2f}")
print(f"Status: {payment_intent.status}")
print(f"Client Secret: {payment_intent.client_secret[:30]}...")

### 2.3 Confirm Payment

In production, the client uses Stripe.js/Elements. Here we simulate with a test card:

In [ ]:
# Create a test payment method and confirm the PaymentIntent
payment_method = stripe.PaymentMethod.create(
    type="card",
    card={
        "number": "4242424242424242",  # Test card
        "exp_month": 12,
        "exp_year": 2027,
        "cvc": "123",
    },
)

# Confirm the payment
confirmed_payment = stripe.PaymentIntent.confirm(
    payment_intent.id,
    payment_method=payment_method.id,
)

print(f"Payment Status: {confirmed_payment.status}")
print(f"Amount captured: ${confirmed_payment.amount_received / 100:.2f}")

if confirmed_payment.status == "succeeded":
    print("\n✓ Payment successful! Funds are now in platform's Stripe balance.")
    print("  Ready to transfer to restaurant and courier.")

---
# Section 3: Routing Funds with Transfers

> **Goal:** Split the $35 payment between restaurant, courier, and platform

### Why Separate Charges & Transfers?

- **Timing control** — Transfer after delivery confirmed, not before
- **Amount control** — Adjust for tips, refunds, promotions
- **Multi-party** — Split to any number of connected accounts
- **Dispute handling** — Platform holds funds, manages disputes centrally

### 3.1 Transfer to Restaurant ($25)

In [ ]:
# Get the charge ID from the successful PaymentIntent
charge_id = confirmed_payment.latest_charge
print(f"Source Charge ID: {charge_id}")

# Transfer food cost to restaurant
restaurant_transfer = stripe.Transfer.create(
    amount=FOOD_COST,  # $25.00
    currency="usd",
    destination=restaurant_account.id,
    source_transaction=charge_id,  # Links transfer to original charge
    metadata={
        "order_id": "order_98765",
        "transfer_type": "food_payment",
    },
    description="Order #98765 - Food payment",
)

print(f"\nRestaurant Transfer ID: {restaurant_transfer.id}")
print(f"Amount: ${restaurant_transfer.amount / 100:.2f}")
print(f"Destination: {restaurant_transfer.destination}")

### 3.2 Transfer to Courier ($7)

In [ ]:
# Transfer delivery fee to courier
courier_transfer = stripe.Transfer.create(
    amount=DELIVERY_FEE,  # $7.00
    currency="usd",
    destination=courier_account.id,
    source_transaction=charge_id,
    metadata={
        "order_id": "order_98765",
        "transfer_type": "delivery_fee",
    },
    description="Order #98765 - Delivery fee",
)

print(f"Courier Transfer ID: {courier_transfer.id}")
print(f"Amount: ${courier_transfer.amount / 100:.2f}")
print(f"Destination: {courier_transfer.destination}")

### 3.3 Verify Fund Distribution

In [ ]:
# Summary of fund distribution
print("=" * 50)
print("FUND DISTRIBUTION SUMMARY")
print("=" * 50)
print(f"\nOrder Total:           ${TOTAL_AMOUNT / 100:.2f}")
print(f"\n  → Restaurant:        ${FOOD_COST / 100:.2f}")
print(f"  → Courier:           ${DELIVERY_FEE / 100:.2f}")
print(f"  → Platform (kept):   ${PLATFORM_FEE / 100:.2f}")
print(f"\n  Total distributed:   ${(FOOD_COST + DELIVERY_FEE + PLATFORM_FEE) / 100:.2f}")
print("=" * 50)

# Verify with transfer list
transfers = stripe.Transfer.list(limit=2)
print("\nRecent Transfers:")
for t in transfers.data:
    print(f"  {t.id}: ${t.amount/100:.2f} → {t.destination}")

---
# Section 4: Edge Cases & Real-World Scenarios

> **Goal:** Handle common platform scenarios like refunds, tips, and instant payouts

### 4.1 Refunds

In [ ]:
# Full refund example (creates a new payment to demonstrate)
# In production, you'd refund the actual order's payment

# Create a refund
# refund = stripe.Refund.create(
#     charge=charge_id,
#     # For partial refund, specify amount:
#     # amount=1500,  # $15.00
# )

print("Refund Considerations:")
print("- Full refund: Reverse all transfers, refund customer")
print("- Partial refund: May need to adjust transfers proportionally")
print("- Transfer reversals: Use stripe.Transfer.create_reversal()")
print("- Timing: Refunds can take 5-10 business days to appear")

### 4.2 Transfer Reversals

Clawing back funds from a connected account:

In [2]:
# Reverse a transfer (partial or full)
# reversal = stripe.Transfer.create_reversal(
#     restaurant_transfer.id,
#     amount=500,  # Reverse $5.00 (partial)
# )

print("Transfer Reversal Use Cases:")
print("- Customer refund requiring funds back from connected account")
print("- Order cancellation before fulfillment")
print("- Dispute/chargeback recovery")
print("")
print("Important: Can only reverse if connected account has sufficient balance")

Transfer Reversal Use Cases:
- Customer refund requiring funds back from connected account
- Order cancellation before fulfillment
- Dispute/chargeback recovery

Important: Can only reverse if connected account has sufficient balance


### 4.3 Tips

In [ ]:
# Option 1: Separate PaymentIntent for tip (recommended)
TIP_AMOUNT = 500  # $5.00 tip

print("Tip Handling Options:")
print("")
print("1. Separate PaymentIntent (Recommended)")
print("   - Create new PaymentIntent for tip amount")
print("   - Transfer 100% of tip to courier")
print("   - Clear audit trail, easy accounting")
print("")
print("2. Include in original order")
print("   - Customer adds tip before payment")
print("   - Adjust transfer amounts accordingly")
print("   - Simpler UX, but tip amount must be known upfront")

### 4.4 Instant Payouts for Couriers

Gig workers often want immediate access to earnings:

In [ ]:
# Check if instant payouts are available
# instant_payout = stripe.Payout.create(
#     amount=700,
#     currency="usd",
#     method="instant",  # instant vs standard
#     stripe_account=courier_account.id,
# )

print("Payout Options:")
print("")
print("Standard Payout:")
print("  - 1-2 business days")
print("  - No additional fee")
print("")
print("Instant Payout:")
print("  - Minutes to debit card")
print("  - 1% fee (min $0.50)")
print("  - Requires eligible debit card")
print("  - Great for gig workers who need immediate access")

---
# Cleanup

Clean up test data created during this demo:

In [ ]:
# Uncomment to delete test accounts (be careful in production!)

# Delete connected accounts
# stripe.Account.delete(restaurant_account.id)
# stripe.Account.delete(courier_account.id)

# Delete customer
# stripe.Customer.delete(customer.id)

print("Test Objects Created:")
print(f"  Restaurant Account: {restaurant_account.id}")
print(f"  Courier Account: {courier_account.id}")
print(f"  Customer: {customer.id}")
print(f"  PaymentIntent: {payment_intent.id}")
print(f"  Restaurant Transfer: {restaurant_transfer.id}")
print(f"  Courier Transfer: {courier_transfer.id}")
print("\nUncomment the delete calls above to clean up.")

---
# Key Takeaways

### Why Separate Charges & Transfers?

| Benefit | Description |
|---------|-------------|
| **Flexibility** | Control when and how much to transfer |
| **Multi-party** | Split payments to any number of recipients |
| **Timing** | Transfer after delivery confirmation, not before |
| **Disputes** | Platform holds funds, manages disputes centrally |

### Connect Account Types

| Feature | Custom | Express | Standard |
|---------|--------|---------|----------|
| Onboarding UI | Your own | Stripe hosted | Stripe hosted |
| Dashboard access | No | Limited | Full |
| Platform responsibility | High | Medium | Low |
| Best for | Marketplaces, delivery | Freelance platforms | SaaS, invoicing |